# Clutch problem

## Defining problem

In [14]:
import numpy as np
from desdeo_problem import ScalarConstraint
from desdeo_problem import _ScalarObjective
from desdeo_problem import variable_builder
from desdeo_problem import MOProblem

from desdeo_emo.EAs.IMOEAD import IMOEA_D

In [5]:
# constants and variables

mu = 0.5
rho = 0.0000078
Mf = 3
n = 250
Jz = 55
w = np.pi * n / 30

def Mh(x):
    x = np.atleast_2d(x)
    return (2 / 3) * mu * x[:, 5 - 2] * x[:, 6 - 2] * ((x[:, 3 - 2] ** 3 - x[:, 2 - 2] ** 3) / (x[:, 3 - 2] ** 2 - x[:, 2 - 2] ** 2)) * 0.001

def th(x):
    x = np.atleast_2d(x)
    return Jz * w / (Mh(x) + Mf)

In [7]:
# objectives

def f_1(x):
    x = np.atleast_2d(x)
    return np.pi * (x[:, 3 - 2]**2 - x[:, 2 - 2]**2) * x[:, 4 - 2] * (x[:, 6 - 2] + 1) * rho

def f_2(x):
    return th(x)

def f_3(x):
    x = np.atleast_2d(x)
    return x[:, 6 - 2]

def f_4(x):
    x = np.atleast_2d(x)
    return x[:, 3 - 2]

def f_5(x):
    x = np.atleast_2d(x)
    return x[:, 5 - 2]

obj1 = _ScalarObjective("obj1", f_1, maximize=True)
obj2 = _ScalarObjective("obj2", f_2, maximize=True)
obj3 = _ScalarObjective("obj3", f_3)
obj4 = _ScalarObjective("obj4", f_4, maximize=True)
obj5 = _ScalarObjective("obj5", f_5, maximize=True)

In [8]:
# decision variables

var_names = ["x1", "x2", "x3", "x4", "x5"]  # Make sure that the variable names are meaningful to you.

initial_values = np.array([70, 100, 2, 800, 5])
lower_bounds = [60, 90, 1, 600, 1]
upper_bounds = [80, 110, 3, 1000, 10]
bounds = np.stack((lower_bounds, upper_bounds))
variables = variable_builder(var_names, initial_values, upper_bounds=upper_bounds, lower_bounds=lower_bounds)

In [9]:
# constraints

Rimin = 60 
Romax = 110 
Amin = 1.5 
deltaR = 20 
Amax = 3 
delta = 0.5 
Lmax = 30 
Zmax = 10
Vsrmax = 10000
# mu = 0.5
# rho = 0.0000078
s = 1.5 
Ms = 40 
# Mf = 3
# n = 250
Pmax = 1   # 10 
# Jz = 55
tmax = 15 
Fmax = 1000 

def S(x):
    x = np.atleast_2d(x)
    return np.pi * (x[:, 3 - 2]**2 - x[:, 2 - 2]**2)

def Prz(x):
    x = np.atleast_2d(x)
    return x[:, 5 - 2] / S(x)

def Rsr(x):
    x = np.atleast_2d(x)
    return (2 / 3) * ((x[:, 3 - 2]**3 - x[:, 2 - 2]**3) / (x[:, 3 - 2]**2 - x[:, 2 - 2]**2))

def Vsr(x):
    x = np.atleast_2d(x)
    return (np.pi * Rsr(x) * n) / 30

# w = pi * n / 30
# Mh = (2 / 3) * mu * x(5 - 1) * x(6 - 1) * ((x(3 - 1) ** 3 - x(2 - 1) ** 3) / (x(3 - 1) ** 2 - x(2 - 1) ** 2)) * 0.001

# c(1) = -x(2 - 1) + Rimin
# c(2) = -Romax + x(3 - 1)
def c_1(x, _):
    x = np.atleast_2d(x)
    return -(x[:, 3 - 2] - x[:, 2 - 2]) + deltaR
 # c(4) = -x(4 - 1) + Amin 
 # c(5) = -Amax + x(4 - 1)

def c_2(x, _):
    x = np.atleast_2d(x)
    return -Lmax + (x[:, 6 - 2] + 1) * (x[:, 4 - 2] + delta)

# c(3) = -Zmax + (x(6 - 1) + 1)
# c(8) = -x(6 - 1) + 1

def c_3(x, _):
    return -Pmax + Prz(x)

def c_4(x, _):
    return -Pmax * Vsrmax + Prz(x) * Vsr(x)

def c_5(x, _):
    return -Vsrmax + Vsr(x)

def c_6(x, _):
    return -tmax + th(x)

def c_7(x, _):
    return -Mh(x) + s * Ms

def c_8(x, _):
    return -th(x)

# c(15) = -x(5 - 1)
# c(16) = -Fmax + x(5 - 1)

cons1 = ScalarConstraint("c_1", 5, 5, c_1)
cons2 = ScalarConstraint("c_2", 5, 5, c_2)
cons3 = ScalarConstraint("c_3", 5, 5, c_3)
cons4 = ScalarConstraint("c_4", 5, 5, c_4)
cons5 = ScalarConstraint("c_5", 5, 5, c_5)
cons6 = ScalarConstraint("c_6", 5, 5, c_6)
cons7 = ScalarConstraint("c_7", 5, 5, c_7)
cons8 = ScalarConstraint("c_8", 5, 5, c_8)

In [15]:
# problem
problem = MOProblem(objectives=[obj1, obj2, obj3, obj4, obj5], variables=variables, constraints=[cons1, cons2, cons3, cons4,
                                                                                         cons5, cons6, cons7, cons8])
evolver = IMOEA_D(problem, n_iterations=1, n_gen_per_iter=300)

plot, pref = evolver.requests()

print(plot.content["dimensions_data"])

print(pref[0].content['message'])

AttributeError: 'NoneType' object has no attribute 'content'